# Physics Informed Neural Networks <br> F1 Car Front Wing Aerodymanics

## PINN

In [1]:
import pandas as pd
import torch
from pinn import PINN
import os

In [2]:
data_dir = "/Users/ggito/repos/pinns/data/front_wing"
model_dir = os.path.join(data_dir, "models")

points_filename = "points_final.csv"
norms_filename = "norms_final.csv"

In [3]:
if torch.backends.mps.is_available():
  device = torch.device("mps")
elif torch.cuda.is_available():
  device = torch.device("cuda")
else:
  print("GPU device not found.")

print(device)

mps


In [4]:
wing_df = pd.read_csv(os.path.join(data_dir, points_filename))
# norm_df = pd.read_csv(os.path.join(in_dir, norms_filename))

In [5]:
# Density (rho): 1.2041kg/m^3
# Dynamic viscosity (mu): 1.81e-5 kg/m.s
rho = 1
mu = 1

# m/s
in_velocity = 1

# Domain limits
x_max = 1
y_max = 1
z_max = 1
t_max = 2

In [6]:
input_dim = 4
output_dim = 4
# hidden_units = [2058, 2058, 2058, 2058, 2058]
# hidden_units = [1024, 1024, 1024, 1024, 1024]
hidden_units = [2058, 2058, 2058]
# hidden_units = [4, 4, 4]

model_name = "v8"

pinn = PINN(input_dim, output_dim, hidden_units, model_name).to(device)

In [7]:
optimizer = torch.optim.LBFGS(pinn.parameters(), lr=1)

epochs = 2000
checkpoint_epochs = 10

Nf = 500    # num of collocation points -> pde evaluation
N0 = 500    # num of points to evaluate initial conditons
Nb = 1500   # num of points to evaluate boundary conditions
Nw = 500    # num of points of the surface of the front wing to evaluate boundary conditions

In [9]:
pinn.train_pinn(
    epochs=epochs, 
    optimizer=optimizer, 
    s_df=wing_df, 
    Nf=Nf, 
    N0=N0, 
    Nb=Nb, 
    Ns=Nw, 
    x_max=x_max, 
    y_max=y_max, 
    z_max=z_max, 
    t_max=t_max, 
    in_velocity=in_velocity, 
    mu=mu, 
    rho=rho, 
    device=device, 
    checkpoint_epochs=checkpoint_epochs, 
    model_dir=model_dir)

PINN(
  (layers): ModuleList(
    (0): Linear(in_features=4, out_features=2058, bias=True)
    (1-2): 2 x Linear(in_features=2058, out_features=2058, bias=True)
    (3): Linear(in_features=2058, out_features=4, bias=True)
  )
)
Model name: v8
Number of epochs: 2000
Number of collocation points Nf: 500
Number of initial condition points N0: 500
Number of boundary condition points Nb: 1500
Number of object surface points Ns: 500
X max: 1, Y max: 1, Z max: 1, T max: 2
mu: 1, rho: 1
c1: 1.0, c2: 1.0, c3: 1.0, c4: 1.0, c5: 1.0, c6: 1.0, c7: 1.0, c8: 1.0, c9: 1.0, c10: 1.0
Inflow velocity: 1
Device: mps
Checkpoint epochs: 10
Model directory: /Users/ggito/repos/pinns/data/front_wing/models
=> converting Nf, N0, Nb, Ns to nearest power of 2...
Nf: 512, N0: 512, Nb: 2048, Ns: 512
=> starting training...


In [8]:
pinn, optimizer = pinn.load_checkpoint_num(optimizer, model_dir, model_name, 40)

=> loading checkpoint '/Users/ggito/repos/pinns/data/front_wing/models/v8/40.pt'
=> loaded checkpoint '/Users/ggito/repos/pinns/data/front_wing/models/v8/40.pt' (epoch 40)
